## Collecting the Lists of Data
Because we are using data spanning 5 year-long periods, the plan is to use a loop to accumulate the data for a given year and then to add each of these dataframes together.

In [77]:
data_17_18 = ['Gradsattendingcollege-17-18.xlsx','enrollmentbyracegender-17-18.xlsx',
              'selectedpopulations-17-18.xlsx','TeacherSalaries-17-18.xlsx',
              'PerPupilExpenditures-17-18.xlsx','StudentDisciplineDataReport-17-18.xlsx']

In [78]:
data_16_17 = ['Gradsattendingcollege-16-17.xlsx','enrollmentbyracegender-16-17.xlsx',
              'selectedpopulations-16-17.xlsx','TeacherSalaries-16-17.xlsx',
              'PerPupilExpenditures-16-17.xlsx','StudentDisciplineDataReport-16-17.xlsx']

In [79]:
data_15_16 = ['Gradsattendingcollege-15-16.xlsx','enrollmentbyracegender-15-16.xlsx',
              'selectedpopulations-15-16.xlsx','TeacherSalaries-15-16.xlsx',
              'PerPupilExpenditures-15-16.xlsx','StudentDisciplineDataReport-15-16.xlsx']

In [80]:
data_14_15 = ['Gradsattendingcollege-14-15.xlsx','enrollmentbyracegender-14-15.xlsx',
              'selectedpopulations-14-15.xlsx','TeacherSalaries-14-15.xlsx',
              'PerPupilExpenditures-14-15.xlsx','StudentDisciplineDataReport-14-15.xlsx']

In [81]:
data_13_14 = ['Gradsattendingcollege-13-14.xlsx','enrollmentbyracegender-13-14.xlsx',
              'selectedpopulations-13-14.xlsx','TeacherSalaries-13-14.xlsx',
              'PerPupilExpenditures-13-14.xlsx','StudentDisciplineDataReport-13-14.xlsx']

In [122]:
giant_list = [data_17_18,data_16_17,data_15_16,data_14_15,data_13_14]

## Creating the Function For Each Year
The 6 xlsx files in a given year will be joined using the district code. The district code is *not* kept after the dataset is compiled since this is not important for our machine learning tactics. However, given that the dataset is being compiled with code it will be easy to create a version with it in case we wish to look at it when understanding our data. 

   #### What We're Keeping:
   - Gradsattendingcollege: (%) Attending College/University (this is the **outcome**)
   - enrollmentbyracegender: (%) Races/Ethnicity (7) and Gender (2) *only added non-binary 2018-19 :(((* 
   - selectedpopulations: (%) First Language Not English, English Language Learner, Students With Disabilities, and High Needs
   - TeacherSalaries: (\$) Salary Totals and Average Salary; (#) FTE count
   - PerPupilExpenditures: (\$) In-District Expenditures, In-District Expenditures per Pupil, Total Expenditures, and Total Expenditures per Pupil; (#) Total In-district FTEs and Total Pupil FTEs
   - StudentDisciplineDataReport: (#) Students and Students Diciplined, (%) In-School Suspension, Out-of-School Suspension, Expulsion, and Removed to Alternate Setting
   
   We did have to drop some columns due to the availability of what was recorded each year.

In [128]:
import pandas as pd
import numpy as np

In [130]:
#initialize our final dataset
big = pd.DataFrame()

# going through the files for each of the 5 years
for year in giant_list:
    for ex in year:
        
        # for the attending college data (outcome!) 
        if ex.startswith('Grads'):
            # get only the index and outcome col
            grads = pd.read_excel(ex,header = 1,index_col="District Code",usecols="B,E")
            
        # for the demographic data
        elif ex.startswith('enroll'):
            enroll = pd.read_excel(ex,header = 1,index_col="District Code",usecols="B:K")
            
        # for other population data
        elif ex.startswith('selected'):
            selec = pd.read_excel(ex,header = 1,index_col="District Code",usecols="B,D,F,H,P")
            # incorrectly named in download
            selec = selec.rename({"High Needs #.1":"High Needs %"}, axis = 1)
            
        # for the Teacher data
        elif ex.startswith('Teacher'):
            teach = pd.read_excel(ex,header = 1,index_col="District Code",usecols="B:E")
            
        # for the expenditures data
        elif ex.startswith('PerPupil'):
            per = pd.read_excel(ex,header = 1,index_col="District Code",usecols="B:H")
            
        # for the student dicipline data
        elif ex.startswith('Student'):
            stu = pd.read_excel(ex,header = 1,index_col="District Code",usecols="B:H")
            
    # merge all the data
    merged = pd.merge(grads,enroll,left_index=True, right_index=True)
    merged = pd.merge(merged,selec,left_index=True, right_index=True)
    merged = pd.merge(merged,teach,left_index=True, right_index=True)
    merged = pd.merge(merged,per,left_index=True, right_index=True)
    merged = pd.merge(merged,stu,left_index=True, right_index=True)
    
    # drop the row '0' which represents the totals for the year
    merged = merged.drop(0)
    
    # dropna
    merged = merged.dropna()
    
    # get rid of the index
    merged.reset_index(drop=True, inplace=True)
    
    # compile the data
    big = big.append(merged)

# drop any na data from the final version
big = big.dropna()

In [132]:
# Outputs the columns in our dataset
big.columns

Index(['Attending Coll./Univ. (%)', 'African American', 'Asian', 'Hispanic',
       'White', 'Native American', 'Native Hawaiian, Pacific Islander',
       'Multi-Race, Non-Hispanic', 'Males', 'Females',
       'First Language Not English %', 'English Language Learner %',
       'Students With Disabilities %', 'High Needs %', 'Salary Totals',
       'Average Salary', 'FTE Count', 'In-District Expenditures',
       'Total In-district FTEs', 'In-District Expenditures per Pupil',
       'Total Expenditures', 'Total Pupil FTEs',
       'Total Expenditures per Pupil', 'Students', 'Students Disciplined',
       '% In-School Suspension', '% Out-of-School Suspension', '% Expulsion',
       '% Removed to Alternate Setting'],
      dtype='object')

In [135]:
# A view of our dataset
big

,Attending Coll./Univ. (%),African American,Asian,Hispanic,White,Native American,"Native Hawaiian, Pacific Islander","Multi-Race, Non-Hispanic",Males,Females,...,In-District Expenditures per Pupil,Total Expenditures,Total Pupil FTEs,Total Expenditures per Pupil,Students,Students Disciplined,% In-School Suspension,% Out-of-School Suspension,% Expulsion,% Removed to Alternate Setting
0,74.0,3.1,2.0,7.9,85.9,0.3,0.1,0.7,50.5,49.5,...,"$13,461.03","$31,082,174.00","2,139.6","$14,527.10","2,122",75,2.3,1.7,0.0,0.0
1,91.8,2.3,32.6,4.5,56.1,0.2,0.1,4.2,51.7,48.3,...,"$14,382.42","$90,445,894.00","5,761.7","$15,697.78","5,726",43,0.3,0.5,0.0,0.0
2,66.3,4.4,0.9,2.5,90.3,0.2,0.1,1.6,50.5,49.5,...,"$15,721.64","$21,089,177.00","1,418.1","$14,871.43","1,282",75,4.1,2.7,0.0,0.0
3,79.3,2.3,3.0,7.8,84.3,0.0,0.1,2.6,52.1,47.9,...,"$15,902.26","$66,245,260.00","3,955.6","$16,747.21","4,002",104,1.5,1.3,0.0,0.0
4,83.2,1.6,1.1,7.2,86.2,0.2,0.1,3.5,53.5,46.5,...,"$15,644.74","$37,327,984.00","2,268.4","$16,455.64","2,227",45,1.4,0.9,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,69.6,1.4,1.8,5.7,88.4,0.4,0.1,2.2,51.8,48.2,...,"$12,440.52","$18,830,817.55","1,534.3","$12,273.23","1,447",93,4.0,3.5,0.0,0.0
245,88.6,1.1,13.8,3.1,77.6,0.2,0.0,4.2,50.2,49.8,...,"$11,835.91","$56,536,546.58","4,494.6","$12,578.77","4,511",66,1.2,0.6,0.0,0.0
246,79.0,1.5,1.0,10.2,85.3,0.3,0.1,1.7,49.4,50.6,...,"$11,893.18","$24,749,632.88","2,010.5","$12,310.19","2,054",74,0.7,3.1,0.0,0.0
247,79.9,6.7,7.6,9.2,73.8,0.4,0.2,2.3,51.0,49.0,...,"$14,162.64","$74,634,007.02","4,956.3","$15,058.41","5,010",142,0.8,2.4,0.0,0.0


In [134]:
# export the dataset as a CSV file!
big.to_csv('education_dataset.csv', index=False)